In [1]:
import pandas as pd
url="http://cssbook.net/d/guns-polls.csv"
d=pd.read_csv(url)
d = d.rename(columns={'Republican Support': 'rep', 'Democratic Support': 'dem'})
d = d.drop(columns='URL')
d2 = d.loc[d.Question == 'arm-teachers']
d2

,Question,Start,End,Pollster,Population,Support,rep,dem
7,arm-teachers,2/23/18,2/25/18,YouGov/Huffpost,Registered Voters,41,69,20
8,arm-teachers,2/20/18,2/23/18,CBS News,Adults,44,68,20
9,arm-teachers,2/27/18,2/28/18,Rasmussen,Adults,43,71,24
10,arm-teachers,2/27/18,2/28/18,NPR/Ipsos,Adults,41,68,18
11,arm-teachers,3/3/18,3/5/18,Quinnipiac,Registered Voters,40,77,10
12,arm-teachers,2/26/18,2/28/18,SurveyMonkey,Registered Voters,43,80,11


In [2]:
d["diff"] = d.rep + d.dem
d = d.assign(diff=d.rep - d.dem)
d.head()

,Question,Start,End,Pollster,Population,Support,rep,dem,diff
0,age-21,2/20/18,2/23/18,CNN/SSRS,Registered Voters,72,61,86,-25
1,age-21,2/27/18,2/28/18,NPR/Ipsos,Adults,82,72,92,-20
2,age-21,3/1/18,3/4/18,Rasmussen,Adults,67,59,76,-17
3,age-21,2/22/18,2/26/18,Harris Interactive,Registered Voters,84,77,92,-15
4,age-21,3/3/18,3/5/18,Quinnipiac,Registered Voters,78,63,93,-30


In [3]:
groups = d.groupby('Question')
groups.agg({'Support': ['mean', 'std']})

Support          
                                  mean       std
Question                                        
age-21                       75.857143  6.011893
arm-teachers                 42.000000  1.549193
background-checks            87.428571  7.322503
ban-assault-weapons          61.750000  6.440285
ban-high-capacity-magazines  67.285714  3.860669
mental-health-own-gun        85.833333  5.455884
repeal-2nd-amendment         10.000000       NaN
stricter-gun-laws            66.454545  5.145165

In [4]:
d.groupby('Question').agg({'Support': ['mean', 'std']})

Support          
                                  mean       std
Question                                        
age-21                       75.857143  6.011893
arm-teachers                 42.000000  1.549193
background-checks            87.428571  7.322503
ban-assault-weapons          61.750000  6.440285
ban-high-capacity-magazines  67.285714  3.860669
mental-health-own-gun        85.833333  5.455884
repeal-2nd-amendment         10.000000       NaN
stricter-gun-laws            66.454545  5.145165

In [5]:
d["mean"] = d.groupby('Question')["Support"].transform('mean')
d["deviation"] = d["Support"] - d["mean"]
d.head()

,Question,Start,End,Pollster,Population,Support,rep,dem,diff,mean,deviation
0,age-21,2/20/18,2/23/18,CNN/SSRS,Registered Voters,72,61,86,-25,75.857143,-3.857143
1,age-21,2/27/18,2/28/18,NPR/Ipsos,Adults,82,72,92,-20,75.857143,6.142857
2,age-21,3/1/18,3/4/18,Rasmussen,Adults,67,59,76,-17,75.857143,-8.857143
3,age-21,2/22/18,2/26/18,Harris Interactive,Registered Voters,84,77,92,-15,75.857143,8.142857
4,age-21,3/3/18,3/5/18,Quinnipiac,Registered Voters,78,63,93,-30,75.857143,2.142857


In [16]:
# version of the guns polls with some errors introduced
url="http://cssbook.net/d/guns-polls-dirty.csv"
dirty=pd.read_csv(url)

# Option 1: clean with direct assignment
# Note that when creating a new column, 
# you have to use df["col"] rather than df.col 
dirty["rep2"] = dirty.rep.str.replace("[^0-9\\.]", "")
dirty["rep2"] = pd.to_numeric(dirty.rep2)
dirty["Support2"] = dirty.Support.fillna(dirty.Support.mean())

# Alternatively, clean with apply 
# Note the need to use an anonymous function
# (lambda) to chain calculations
cleaned = dirty.assign(
    rep2 = dirty.rep.str.replace("[^0-9\\.]", ""),
    rep3 = lambda d: pd.to_numeric(d.rep2),
    Support2 = dirty.Support.fillna(dirty.Support.mean()))

# Finally, you can create your own function 
import re
def clean_num(x):
    x = re.sub("[^0-9\\.]", "", x)
    return int(x)

cleaned["rep3"] = cleaned.rep.apply(clean_num)
cleaned.head()

,Question,Start,End,Pollster,Population,Support,rep,dem,rep2,Support2,rep3
0,arm-teachers,2/23/18,2/25/18,YouGov/Huffpost,Registered Voters,41.0,69,20,69,41.0,69
1,arm-teachers,2/20/18,2/23/18,CBS News,Adults,NaN,68,20,68,41.6,68
2,arm-teachers,2/27/18,2/28/18,Rasmussen,Adults,43.0,71d,24,71,43.0,71
3,arm-teachers,2/27/18,2/28/18,NPR/Ipsos,Adults,41.0,68,18,68,41.0,68
4,arm-teachers,3/3/18,3/5/18,Quinnipiac,Registered Voters,40.0,77,10,77,40.0,77


In [ ]:
dirty.Support.mean()